### 173 original month_lag で期間を区切り集計を行う
相対的に時系列のズレはあるが、初期3ヶ月のトランザクションを共通集計し、targetの期間と一致すると考えられるグループで予測モデルを作り、他のグループはその予測モデルで初回のロイヤリティを予測する

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)

df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)

100%|██████████| 3/3 [00:22<00:00,  7.50s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(29112361, 21)
(29112361, 22)


In [22]:
def feat_agg(df, fname):
    
    global train_test
    
    df = pd.get_dummies(df, columns=['category_2', 'category_3'])
    
#     aggs = {}
    if fname.count('cat1'):
        aggs = {
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    else:
        aggs = {
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    
    if not(fname.count('cat')):
        aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
#     df_agg.drop(f'{fname}_all_term_max', axis=1, inplace=True)
    
    # Monthly Agg
#     del aggs['all_term']
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    month_agg.reset_index(drop=False,inplace=True)
    
    train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
    del df_agg, month_agg
    gc.collect()
    
    
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


def save_feature(col):
    ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
    if col in ignore_features: return
    if (col.count('feature_')):return
    if (col.count('purchase_date')):return
    feature = train_test[col].fillna(-1).astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)


def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df

In [27]:
# term
term_num = 5
# base_list = np.zeros(term_num).astype('int8')
base_list = np.arange(1, term_num+1, 1)
last_list = np.arange(1, term_num+1, 1)+2
col_month = 'org_month_lag'
cat1_0 = False
# cat1_0 = True
feat_list = []

for base_month, last_month in zip(base_list, last_list):
    
    term_type = f'org_lag{base_month}_{last_month}'
    
    df = df_hist[df_hist[col_month]<=last_month]
    df = df[df[col_month]>=base_month]
    
    auth1 = df[df.authorized_flag==1]
    auth0 = df[df.authorized_flag==0]

    if cat1_0:
        df_list = [
            auth1[auth1.category_1==1]
            ,auth1[auth1.category_1==0]
            ,auth0[auth0.category_1==1]
            ,auth0[auth0.category_1==0]
        ]
        fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0']
    else:
        df_list = [auth1, auth0]
        fname_list = [f'{term_type}_auth1', f'{term_type}_auth0']
        
        
    for df, fname in zip(df_list, fname_list):
        
        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        df_train.set_index(key, inplace=True)
        df_test.set_index(key, inplace=True)
        train_test = pd.concat([df_train, df_test], axis=0)
        
        feat_agg(df, fname)
    
        train_test['first_active_month'] =  pd.to_datetime(train_test['first_active_month'])
        for col in train_test.columns:
            if col.count('purchase_date'):
                train_test[col] = pd.to_datetime( train_test[col])
        
        # 最終までの期間
        train_test[f'{fname}_term_from_first_month'] = (train_test[f'{fname}_purchase_date_max'] - train_test[f'first_active_month']).dt.days
        
        # 20180501までの期間を各データセットパターンで
        train_test[f'{fname}_term_from_now'] = (pd.to_datetime('2018-05-01') - train_test[f'{fname}_purchase_date_max']).dt.days
        
        # auth1, auth0の間の期間
        # とりあえずいらない
#         if cat1_0:
#             train_test[f'{term_type}_auth1_0_cat0_term'] = train_test[f'{term_type}_auth1_cat0_term_from_now'] - train_test[f'{term_type}_auth0_cat0_term_from_now']
#             train_test[f'{term_type}_auth1_0_cat1_term'] = train_test[f'{term_type}_auth1_cat1_term_from_now'] - train_test[f'{term_type}_auth0_cat1_term_from_now']
#         else:
#             train_test[f'{term_type}_auth1_0_term'] = train_test[f'{term_type}_auth1_term_from_now'] - train_test[f'{term_type}_auth0_term_from_now']
#             train_test[f'{term_type}_auth1_0_term'] = train_test[f'{term_type}_auth1_term_from_now'] - train_test[f'{term_type}_auth0_term_from_now']
#             train_test[f'{term_type}_auth1_0_purchase_date_max_diff'] = (train_test[f'{term_type}_auth1_purchase_date_max'] - train_test[f'{term_type}_auth0_purchase_date_max']).dt.days
#             train_test[f'{term_type}_auth1_0_purchase_date_min_diff'] = (train_test[f'{term_type}_auth1_purchase_date_min'] - train_test[f'{term_type}_auth0_purchase_date_min']).dt.days
        
        date_min_cols = [col for col in train_test.columns if col.count('purchase_date_min') and not(col.count('diff'))]
        for col in date_min_cols:
            if cat1_0:
                if col.count('auth1') and col.count('cat0'):
                    train_test[f'{fname}_first_buy'] = (train_test[col] - train_test[f'first_active_month']).dt.days
            else:
                if col.count('auth1'):
                    train_test[f'{fname}_first_buy'] = (train_test[col] - train_test[f'first_active_month']).dt.days

        train_test = impute_agg(train_test)
            
        print(f"Saving features... {fname} {train_test.shape}")
        
        ignore_features = ['first_active_month', 'card_id', target]

        train_test.columns = [col.replace('.', '_') for col in train_test.columns]
        
        ignore_features = ['first_active_month', 'card_id', target]
        
        prefix = f'173_l{base_month}{last_month}'
        print(prefix)
        display(train_test.head(1))
        for col in train_test.columns:
            if col in ignore_features: continue
            if (col.count('feature_')):continue
            if (col.count('unix_date')):continue
            if (col.count('purchase_date')) and not(col.count('diff')):continue
            feature = train_test[col].fillna(-1).astype('float32').values
                
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
#             utils.to_pkl_gzip(path = f'../features/2_second_valid/{prefix}_{col}@', obj=feature)

100%|██████████| 3/3 [00:00<00:00, 179.33it/s]


Saving features... org_lag1_3_auth1 (325540, 76)
173_l13


,feature_1,feature_2,feature_3,first_active_month,target,org_lag1_3_auth1_category_2_1_0_mean,org_lag1_3_auth1_category_2_2_0_mean,org_lag1_3_auth1_category_2_3_0_mean,org_lag1_3_auth1_category_2_4_0_mean,org_lag1_3_auth1_category_2_5_0_mean,org_lag1_3_auth1_category_3_A_mean,org_lag1_3_auth1_category_3_B_mean,org_lag1_3_auth1_category_3_C_mean,org_lag1_3_auth1_category_1_sum,org_lag1_3_auth1_category_1_mean,org_lag1_3_auth1_month_lag_mean,org_lag1_3_auth1_month_lag_std,org_lag1_3_auth1_yyyy_week_nunique,org_lag1_3_auth1_purchase_date_max,org_lag1_3_auth1_purchase_date_min,org_lag1_3_auth1_month_diff_mean,org_lag1_3_auth1_month_diff_std,org_lag1_3_auth1_purchase_amount_sum,org_lag1_3_auth1_purchase_amount_max,org_lag1_3_auth1_purchase_amount_min,org_lag1_3_auth1_purchase_amount_mean,org_lag1_3_auth1_installments_sum,org_lag1_3_auth1_installments_max,org_lag1_3_auth1_installments_min,org_lag1_3_auth1_installments_mean,org_lag1_3_auth1_installments_std,org_lag1_3_auth1_merchant_id_nunique,org_lag1_3_auth1_merchant_category_id_nunique,org_lag1_3_auth1_card_id_size,org_lag1_3_auth1_city_id_nunique,org_lag1_3_auth1_state_id_nunique,org_lag1_3_auth1_subsector_id_nunique,org_lag1_3_auth1_this_term,org_lag1_3_auth1_freq_per_this_term,org_lag1_3_auth1_amount_per_this_term,org_lag1_3_auth1_instthisments_per_this_term,org_lag1_3_auth1_amount_per_installments_sum,org_lag1_3_auth1_amount_per_installments_mean,org_lag1_3_auth1_monthly_avg_category_2_1_0_mean,org_lag1_3_auth1_monthly_avg_category_2_2_0_mean,org_lag1_3_auth1_monthly_avg_category_2_3_0_mean,org_lag1_3_auth1_monthly_avg_category_2_4_0_mean,org_lag1_3_auth1_monthly_avg_category_2_5_0_mean,org_lag1_3_auth1_monthly_avg_category_3_A_mean,org_lag1_3_auth1_monthly_avg_category_3_B_mean,org_lag1_3_auth1_monthly_avg_category_3_C_mean,org_lag1_3_auth1_monthly_avg_category_1_sum,org_lag1_3_auth1_monthly_avg_category_1_mean,org_lag1_3_auth1_monthly_avg_month_lag_mean,org_lag1_3_auth1_monthly_avg_month_lag_std,org_lag1_3_auth1_monthly_avg_yyyy_week_nunique,org_lag1_3_auth1_monthly_avg_month_diff_mean,org_lag1_3_auth1_monthly_avg_month_diff_std,org_lag1_3_auth1_monthly_avg_purchase_amount_sum,org_lag1_3_auth1_monthly_avg_purchase_amount_max,org_lag1_3_auth1_monthly_avg_purchase_amount_min,org_lag1_3_auth1_monthly_avg_purchase_amount_mean,org_lag1_3_auth1_monthly_avg_installments_sum,org_lag1_3_auth1_monthly_avg_installments_max,org_lag1_3_auth1_monthly_avg_installments_min,org_lag1_3_auth1_monthly_avg_installments_mean,org_lag1_3_auth1_monthly_avg_installments_std,org_lag1_3_auth1_monthly_avg_merchant_id_nunique,org_lag1_3_auth1_monthly_avg_merchant_category_id_nunique,org_lag1_3_auth1_monthly_avg_card_id_size,org_lag1_3_auth1_monthly_avg_city_id_nunique,org_lag1_3_auth1_monthly_avg_state_id_nunique,org_lag1_3_auth1_monthly_avg_subsector_id_nunique,org_lag1_3_auth1_term_from_first_month,org_lag1_3_auth1_term_from_now,org_lag1_3_auth1_first_buy
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06-01,-0.820283,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-6.271028,0.746945,14.0,2017-09-30 20:06:30,2017-07-01 09:02:20,267.037383,24.319386,7.347656,0.439697,0.007523,0.068665,0.0,0.0,0.0,0.0,0.0,41.0,16.0,107.0,3.0,1.0,10.0,91.0,1.163043,0.079866,0.0,7.347656,0.068665,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-6.0,0.0,5.333333,259.007091,8.529057,2.449219,0.286621,0.010529,0.076172,0.0,0.0,0.0,0.0,0.0,19.0,9.666667,35.666667,1.666667,1.0,6.0,121.0,212.0,30.0


100%|██████████| 3/3 [00:00<00:00, 178.96it/s]


Saving features... org_lag1_3_auth0 (325540, 75)
173_l13


...

Saving features... org_lag5_7_auth1 (325540, 76)
173_l57


,feature_1,feature_2,feature_3,first_active_month,target,org_lag5_7_auth1_category_2_1_0_mean,org_lag5_7_auth1_category_2_2_0_mean,org_lag5_7_auth1_category_2_3_0_mean,org_lag5_7_auth1_category_2_4_0_mean,org_lag5_7_auth1_category_2_5_0_mean,org_lag5_7_auth1_category_3_A_mean,org_lag5_7_auth1_category_3_B_mean,org_lag5_7_auth1_category_3_C_mean,org_lag5_7_auth1_category_1_sum,org_lag5_7_auth1_category_1_mean,org_lag5_7_auth1_month_lag_mean,org_lag5_7_auth1_month_lag_std,org_lag5_7_auth1_yyyy_week_nunique,org_lag5_7_auth1_purchase_date_max,org_lag5_7_auth1_purchase_date_min,org_lag5_7_auth1_month_diff_mean,org_lag5_7_auth1_month_diff_std,org_lag5_7_auth1_purchase_amount_sum,org_lag5_7_auth1_purchase_amount_max,org_lag5_7_auth1_purchase_amount_min,org_lag5_7_auth1_purchase_amount_mean,org_lag5_7_auth1_installments_sum,org_lag5_7_auth1_installments_max,org_lag5_7_auth1_installments_min,org_lag5_7_auth1_installments_mean,org_lag5_7_auth1_installments_std,org_lag5_7_auth1_merchant_id_nunique,org_lag5_7_auth1_merchant_category_id_nunique,org_lag5_7_auth1_card_id_size,org_lag5_7_auth1_city_id_nunique,org_lag5_7_auth1_state_id_nunique,org_lag5_7_auth1_subsector_id_nunique,org_lag5_7_auth1_this_term,org_lag5_7_auth1_freq_per_this_term,org_lag5_7_auth1_amount_per_this_term,org_lag5_7_auth1_instthisments_per_this_term,org_lag5_7_auth1_amount_per_installments_sum,org_lag5_7_auth1_amount_per_installments_mean,org_lag5_7_auth1_monthly_avg_category_2_1_0_mean,org_lag5_7_auth1_monthly_avg_category_2_2_0_mean,org_lag5_7_auth1_monthly_avg_category_2_3_0_mean,org_lag5_7_auth1_monthly_avg_category_2_4_0_mean,org_lag5_7_auth1_monthly_avg_category_2_5_0_mean,org_lag5_7_auth1_monthly_avg_category_3_A_mean,org_lag5_7_auth1_monthly_avg_category_3_B_mean,org_lag5_7_auth1_monthly_avg_category_3_C_mean,org_lag5_7_auth1_monthly_avg_category_1_sum,org_lag5_7_auth1_monthly_avg_category_1_mean,org_lag5_7_auth1_monthly_avg_month_lag_mean,org_lag5_7_auth1_monthly_avg_month_lag_std,org_lag5_7_auth1_monthly_avg_yyyy_week_nunique,org_lag5_7_auth1_monthly_avg_month_diff_mean,org_lag5_7_auth1_monthly_avg_month_diff_std,org_lag5_7_auth1_monthly_avg_purchase_amount_sum,org_lag5_7_auth1_monthly_avg_purchase_amount_max,org_lag5_7_auth1_monthly_avg_purchase_amount_min,org_lag5_7_auth1_monthly_avg_purchase_amount_mean,org_lag5_7_auth1_monthly_avg_installments_sum,org_lag5_7_auth1_monthly_avg_installments_max,org_lag5_7_auth1_monthly_avg_installments_min,org_lag5_7_auth1_monthly_avg_installments_mean,org_lag5_7_auth1_monthly_avg_installments_std,org_lag5_7_auth1_monthly_avg_merchant_id_nunique,org_lag5_7_auth1_monthly_avg_merchant_category_id_nunique,org_lag5_7_auth1_monthly_avg_card_id_size,org_lag5_7_auth1_monthly_avg_city_id_nunique,org_lag5_7_auth1_monthly_avg_state_id_nunique,org_lag5_7_auth1_monthly_avg_subsector_id_nunique,org_lag5_7_auth1_term_from_first_month,org_lag5_7_auth1_term_from_now,org_lag5_7_auth1_first_buy
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06-01,-0.820283,0.968421,0.0,0.0,0.0,0.031579,1.0,0.0,0.0,0.0,0.0,-1.978947,0.651985,14.0,2018-01-30 12:10:14,2017-11-03 12:30:53,132.684211,20.228095,15.179688,2.0,0.013084,0.15979,0.0,0.0,0.0,0.0,0.0,53.0,29.0,95.0,4.0,3.0,18.0,87.0,1.079545,0.172496,0.0,15.179688,0.15979,0.952381,0.0,0.0,0.0,0.047619,1.0,0.0,0.0,0.0,0.0,-2.0,0.0,5.0,132.859436,8.485247,5.058594,0.933105,0.01387,0.133789,0.0,0.0,0.0,0.0,0.0,23.333333,16.0,31.666667,2.0,1.666667,11.0,243.0,90.0,155.0


100%|██████████| 3/3 [00:00<00:00, 181.36it/s]


Saving features... org_lag5_7_auth0 (325540, 75)
173_l57


,feature_1,feature_2,feature_3,first_active_month,target,org_lag5_7_auth0_category_2_1_0_mean,org_lag5_7_auth0_category_2_2_0_mean,org_lag5_7_auth0_category_2_3_0_mean,org_lag5_7_auth0_category_2_4_0_mean,org_lag5_7_auth0_category_2_5_0_mean,org_lag5_7_auth0_category_3_A_mean,org_lag5_7_auth0_category_3_B_mean,org_lag5_7_auth0_category_3_C_mean,org_lag5_7_auth0_category_1_sum,org_lag5_7_auth0_category_1_mean,org_lag5_7_auth0_month_lag_mean,org_lag5_7_auth0_month_lag_std,org_lag5_7_auth0_yyyy_week_nunique,org_lag5_7_auth0_purchase_date_max,org_lag5_7_auth0_purchase_date_min,org_lag5_7_auth0_month_diff_mean,org_lag5_7_auth0_month_diff_std,org_lag5_7_auth0_purchase_amount_sum,org_lag5_7_auth0_purchase_amount_max,org_lag5_7_auth0_purchase_amount_min,org_lag5_7_auth0_purchase_amount_mean,org_lag5_7_auth0_installments_sum,org_lag5_7_auth0_installments_max,org_lag5_7_auth0_installments_min,org_lag5_7_auth0_installments_mean,org_lag5_7_auth0_installments_std,org_lag5_7_auth0_merchant_id_nunique,org_lag5_7_auth0_merchant_category_id_nunique,org_lag5_7_auth0_card_id_size,org_lag5_7_auth0_city_id_nunique,org_lag5_7_auth0_state_id_nunique,org_lag5_7_auth0_subsector_id_nunique,org_lag5_7_auth0_this_term,org_lag5_7_auth0_freq_per_this_term,org_lag5_7_auth0_amount_per_this_term,org_lag5_7_auth0_instthisments_per_this_term,org_lag5_7_auth0_amount_per_installments_sum,org_lag5_7_auth0_amount_per_installments_mean,org_lag5_7_auth0_monthly_avg_category_2_1_0_mean,org_lag5_7_auth0_monthly_avg_category_2_2_0_mean,org_lag5_7_auth0_monthly_avg_category_2_3_0_mean,org_lag5_7_auth0_monthly_avg_category_2_4_0_mean,org_lag5_7_auth0_monthly_avg_category_2_5_0_mean,org_lag5_7_auth0_monthly_avg_category_3_A_mean,org_lag5_7_auth0_monthly_avg_category_3_B_mean,org_lag5_7_auth0_monthly_avg_category_3_C_mean,org_lag5_7_auth0_monthly_avg_category_1_sum,org_lag5_7_auth0_monthly_avg_category_1_mean,org_lag5_7_auth0_monthly_avg_month_lag_mean,org_lag5_7_auth0_monthly_avg_month_lag_std,org_lag5_7_auth0_monthly_avg_yyyy_week_nunique,org_lag5_7_auth0_monthly_avg_month_diff_mean,org_lag5_7_auth0_monthly_avg_month_diff_std,org_lag5_7_auth0_monthly_avg_purchase_amount_sum,org_lag5_7_auth0_monthly_avg_purchase_amount_max,org_lag5_7_auth0_monthly_avg_purchase_amount_min,org_lag5_7_auth0_monthly_avg_purchase_amount_mean,org_lag5_7_auth0_monthly_avg_installments_sum,org_lag5_7_auth0_monthly_avg_installments_max,org_lag5_7_auth0_monthly_avg_installments_min,org_lag5_7_auth0_monthly_avg_installments_mean,org_lag5_7_auth0_monthly_avg_installments_std,org_lag5_7_auth0_monthly_avg_merchant_id_nunique,org_lag5_7_auth0_monthly_avg_merchant_category_id_nunique,org_lag5_7_auth0_monthly_avg_card_id_size,org_lag5_7_auth0_monthly_avg_city_id_nunique,org_lag5_7_auth0_monthly_avg_state_id_nunique,org_lag5_7_auth0_monthly_avg_subsector_id_nunique,org_lag5_7_auth0_term_from_first_month,org_lag5_7_auth0_term_from_now
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06-01,-0.820283,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.5,0.57735,3.0,2017-12-27 14:33:33,2017-11-04 14:17:22,152.25,28.721363,0.416992,0.270508,0.018036,0.104248,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,1.0,1.0,4.0,53.0,0.074074,0.007722,0.0,0.416992,0.104248,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.5,0.0,1.5,152.25,2.474874,0.208496,0.151123,0.057434,0.104248,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,2.0,209.0,124.0


### Save Feature

In [28]:
#========================================================================
# City_id
#========================================================================

# term
term_num = 5
# base_list = np.zeros(term_num).astype('int8')
base_list = np.arange(1, term_num+1, 1)
last_list = np.arange(1, term_num+1, 1)+2
col_month = 'org_month_lag'
cat1_0 = False
# cat1_0 = True
feat_list = []

for base_month, last_month in zip(base_list, last_list):
    
    prefix = f'174_l{base_month}{last_month}'
    
    df = df_hist[df_hist[col_month]<=last_month]
    df = df[df[col_month]>=base_month]
    df = df[df.authorized_flag==1]
    
    df['cnt'] = 1
    df_city = df.groupby([key, 'city_id'])['cnt'].sum().reset_index()
    df_rec = df.groupby([key])['cnt'].sum().reset_index()
    
    main = df_city.merge(df_rec, how='inner', on=key)
    del df_city, df_rec
    gc.collect()
    
    auth_mean = df.groupby('city_id')['authorized_flag'].mean()
    
    main['main_city_ratio'] = main['cnt_x'] / (main['cnt_y'])
    main_city_idx = main.groupby(key)['main_city_ratio'].idxmax()
    
    main_city = main.iloc[main_city_idx][[key, 'city_id', 'main_city_ratio']]
    main_city['main_city_auth_mean'] = main_city['city_id'].map(auth_mean)
    main_city.set_index(key, inplace=True)
    
    # load base train test
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    train_test = train_test.join(main_city)
    
    for col in train_test.columns:
        if col in ignore_list:continue
        if not(col.count('main_city_ratio')):continue
            
        feature = train_test[col].fillna(-1).astype('float32').values
        
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
#         utils.to_pkl_gzip(path = f'../features/2_second_valid/{feat_no}_{col}@', obj=feature)

100%|██████████| 3/3 [00:00<00:00, 181.04it/s]
